In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from google_trans_new import google_translator  
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
fg_df = pd.read_csv('D:/Users/ehern/Desktop/Machine Learning/Project/fall_guys_metacritic_reviews.csv')

fg_df.head()

,username,review_type,published_date,score,votes,review_text,profile_url,platform
0,Fen1ma,user,"Aug 26, 2020",6.0,0.0,"на данный момент не советую покупать игру, ещё...",https://www.metacritic.com/user/Fen1ma,pc
1,Locke_VI,user,"Aug 25, 2020",3.0,0.0,Fall Guys was a game I was really looking forw...,https://www.metacritic.com/user/Locke_VI,pc
2,Crisener,user,"Aug 25, 2020",8.0,0.0,You gotta love it. The aesthetics are very col...,https://www.metacritic.com/user/Crisener,pc
3,EnigmaXtreme,user,"Aug 25, 2020",7.0,0.0,When we covered Fall Guys last week during it’...,https://www.metacritic.com/user/EnigmaXtreme,pc
4,nitorita,user,"Aug 23, 2020",7.0,0.0,I like how Fall Guys adapted Mario Party and m...,https://www.metacritic.com/user/nitorita,pc


In [3]:
fg_df.drop(columns=['username','review_type','published_date','published_date', 'votes', 'profile_url', 'platform'],inplace=True)

In [4]:
noNull = fg_df.dropna()

In [26]:
for i in noNull['review_text']:
    detector = google_translator()  
    detect_result = detector.detect(i)
    print(detect_result)

['ru', 'russian']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['es', 'spanish']
['es', 'spanish']
['es', 'spanish']
['es', 'spanish']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['es', 'spanish']
['es', 'spanish']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['pt', 'portuguese']
['en', 'english']
['pt', 'portuguese']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['es', 'spanish']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['ru', 'russian']
['en', 'english']
['en', 'english']
['en', 'english']
['es', 'spanish']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en', 'english']
['en

In [6]:
# translations = {}
# for i in noNull['review_text']:
#     translator = google_translator()
#     translations[i] = translator.translate(i, lang_tgt='en')
    
# noNull.replace(translations, inplace = True)

# # check translation
# noNull

In [7]:
translated_df = noNull.copy()

In [8]:
translated_df.isnull().sum()

score          0
review_text    0
dtype: int64

In [9]:
translated_df.describe()

,score
count,329.000000
mean,6.802432
std,3.340512
min,0.000000
25%,5.000000
50%,8.000000
75%,10.000000
max,10.000000


In [10]:
translated_df['score'].value_counts()

10.0    83
8.0     61
9.0     50
0.0     34
7.0     26
6.0     20
5.0     15
2.0     12
1.0     11
3.0     10
4.0      7
Name: score, dtype: int64

In [11]:
translated_df['review_text'] = translated_df['review_text'].str.replace('[^\w\s]','')

In [12]:
translated_df['review_text'] = translated_df['review_text'].str.lower()

In [13]:
translated_df.head()

,score,review_text
0,6.0,на данный момент не советую покупать игру ещё ...
1,3.0,fall guys was a game i was really looking forw...
2,8.0,you gotta love it the aesthetics are very colo...
3,7.0,when we covered fall guys last week during its...
4,7.0,i like how fall guys adapted mario party and m...


In [14]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
def tokenize(text):
    tokens = re.split('\W+', str(text))
    return tokens

translated_df['review_text'] = translated_df['review_text'].apply(lambda w: tokenize(w))
translated_df.head()

,score,review_text
0,6.0,"[на, данный, момент, не, советую, покупать, иг..."
1,3.0,"[fall, guys, was, a, game, i, was, really, loo..."
2,8.0,"[you, gotta, love, it, the, aesthetics, are, v..."
3,7.0,"[when, we, covered, fall, guys, last, week, du..."
4,7.0,"[i, like, how, fall, guys, adapted, mario, par..."


In [16]:
stopwords = nltk.corpus.stopwords.words('english')
def removeStopWords(text):
    cleanText = [w for w in text if w not in stopwords]
    return cleanText

translated_df['review_text'] = translated_df['review_text'].apply(lambda w: removeStopWords(w))
translated_df.head()

,score,review_text
0,6.0,"[на, данный, момент, не, советую, покупать, иг..."
1,3.0,"[fall, guys, game, really, looking, forward, f..."
2,8.0,"[gotta, love, aesthetics, colorful, charming, ..."
3,7.0,"[covered, fall, guys, last, week, beta, talked..."
4,7.0,"[like, fall, guys, adapted, mario, party, made..."


In [17]:
def listToString(text): 
    str1 = " " 
    return (str1.join(text))
        
translated_df['review_text'] = translated_df['review_text'].apply(lambda w: listToString(w))
translated_df.head()

,score,review_text
0,6.0,на данный момент не советую покупать игру ещё ...
1,3.0,fall guys game really looking forward first an...
2,8.0,gotta love aesthetics colorful charming gamepl...
3,7.0,covered fall guys last week beta talked fun ga...
4,7.0,like fall guys adapted mario party made suppor...


In [18]:
vectorizer = TfidfVectorizer()

In [19]:
X = vectorizer.fit_transform(translated_df['review_text'])
y = translated_df['score']

In [20]:
X

<329x3835 sparse matrix of type '<class 'numpy.float64'>'
	with 11087 stored elements in Compressed Sparse Row format>

In [21]:
y

0      6.0
1      3.0
2      8.0
3      7.0
4      7.0
      ... 
349    1.0
350    9.0
351    8.0
352    9.0
353    9.0
Name: score, Length: 329, dtype: float64

In [22]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)

my_DecisionTree = DecisionTreeClassifier()

my_DecisionTree.fit(X_train, y_train)

y_predict_dt = my_DecisionTree.predict(X_test)

accuracy = accuracy_score(y_test, y_predict_dt)

print(accuracy)

0.42424242424242425


In [23]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predict_dt))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         2
         1.0       1.00      0.50      0.67         2
         2.0       0.00      0.00      0.00         1
         3.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1
         5.0       1.00      0.50      0.67         2
         6.0       0.50      0.50      0.50         2
         7.0       0.00      0.00      0.00         2
         8.0       0.20      0.67      0.31         3
         9.0       0.50      0.20      0.29         5
        10.0       0.57      0.67      0.62        12

    accuracy                           0.42        33
   macro avg       0.34      0.28      0.28        33
weighted avg       0.45      0.42      0.41        33



C:\Users\ehern\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
print(confusion_matrix(y_test, y_predict_dt))

[[0 0 1 0 0 0 0 0 0 0 1]
 [1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 2 0 1]
 [0 0 0 0 0 0 0 0 4 1 0]
 [1 0 0 0 0 0 0 0 2 1 8]]
